In [3]:
from lxml import etree
import numpy as np

In [139]:
# spherical along x, and y, with revolute springs along all diagonals

skyweave_mass = 2.1  # kg

z_offset = 1.2  # m

num_elements_x = 10
length_x = 0.8 # m
x_sep = length_x / (num_elements_x - 1) # the separation between each consecutive mass element in the x axis 

num_elements_y = 10
length_y = 0.8  # m
y_sep = length_y / (num_elements_y - 1) # the separation between each consecutive mass element in the y axis

stiffness = 2
damping = 0

create_diag = True

uni_joint_x_stiffness = stiffness  # N/rad
uni_joint_y_stiffness = stiffness  # N/rad
uni_joint_x_damping = damping  # N/(rad/s)
uni_joint_y_damping = damping  # N/(rad/s)

num_elements = num_elements_x * num_elements_y
element_mass = skyweave_mass / num_elements

sdf = etree.Element("sdf", version="1.7")
robot = etree.SubElement(sdf, "model", name="spring_damper_mesh")
# pose = etree.SubElement(robot, "pose")
# pose.text = "0 0 0 0 0 0"
static = etree.SubElement(robot, "static")
static.text = "false"

# create a grid of point masses
pt_masses = []
for idx_y in range(num_elements_y):
    pt_masses.append([])
    for idx_x in range(num_elements_x):
        m = etree.SubElement(
            robot,
            "link",
            name=f"mass_x{idx_x}_y{idx_y}",
        )
        pose = etree.SubElement(m, "pose")
        pose.text = f"{(idx_x)*x_sep} {(idx_y)*y_sep} {z_offset} 0 0 0"
 
        inertial = etree.SubElement(m, "inertial")
        mass = etree.SubElement(inertial, "mass")
        mass.text = str(element_mass)
        inertia = etree.SubElement(inertial, "inertia")

        # define all inertia components and their values
        inertia_values = {
            "ixx": "1e-5", "ixy": "0", "ixz": "0",
            "iyy": "1e-5", "iyz": "0",
            "izz": "1e-5"
        }

        # create them in a loop
        for name, value in inertia_values.items():
            etree.SubElement(inertia, name).text = value

        collision = etree.SubElement(m, "collision", name=f"collision_x{idx_x}_y{idx_y}")
        geometry = etree.SubElement(collision, "geometry")
        sphere = etree.SubElement(geometry, "sphere")
        radius = etree.SubElement(sphere, "radius")
        radius.text = "0.001"

        visual = etree.SubElement(m, "visual", name=f"visual_x{idx_x}_y{idx_y}")
        geometry_v = etree.SubElement(visual, "geometry")
        sphere_v = etree.SubElement(geometry_v, "box")
        size_v = etree.SubElement(sphere_v, "size")
        size_v.text = "0.015 0.015 0.015"


        pt_masses[idx_y].append(m)

num_springs_dampers = 0
# link adjacent point masses with xy universal joints
for idx_y in range(num_elements_y):
    for idx_x in range(num_elements_x):
        # check if north connection is possible, if so create joint
        ## what are the conditions for a north connection to be possible?
        if idx_x < num_elements_x - 1:
            joint_north = etree.SubElement(
                robot,
                "joint",
                name=f"N_x{idx_x}_y{idx_y}_to_x{idx_x+1}_y{idx_y}",
                type="ball",
            )

            parent = etree.SubElement(joint_north, "parent")
            parent.text = f"mass_x{idx_x}_y{idx_y}"
            child = etree.SubElement(joint_north, "child")
            child.text = f"mass_x{idx_x+1}_y{idx_y}"

            pose = etree.SubElement(joint_north, "pose", relative_to=f"mass_x{idx_x+1}_y{idx_y}")
            pose.text = f"{-0.5*x_sep} 0 0 0 0 0"

        # check if west connection is possible, if so, create joint
        if idx_y < num_elements_y - 1:
            joint_west = etree.SubElement(
                robot,
                "joint",
                name=f"W_x{idx_x}_y{idx_y}_to_x{idx_x}_y{idx_y+1}",
                type="ball",
            )

            parent = etree.SubElement(joint_west, "parent")
            parent.text = f"mass_x{idx_x}_y{idx_y}"
            child = etree.SubElement(joint_west, "child")
            child.text = f"mass_x{idx_x}_y{idx_y+1}"

            pose = etree.SubElement(joint_west, "pose", relative_to=f"mass_x{idx_x}_y{idx_y+1}")
            pose.text = f"0 {-0.5*y_sep} 0 0 0 {np.pi/2}"

        # link diagonal point masses with shear springs attached to ball joints to prevent shear
        # # check if idx_x is even, and idx y is odd, or if idx_x is odd and idx_y is even
        if (idx_x + idx_y) % 2 == 1 or True: # NOTE we are not doing this anymore, as we want all diagonal connections
            # check if south west joint is possible
            if idx_x > 0 and idx_y < num_elements_y - 1 and create_diag:
                num_springs_dampers += 1
                joint_sw = etree.SubElement(
                    robot,
                    "joint",
                    name=f"SW_x{idx_x}_y{idx_y}_to_x{idx_x-1}_y{idx_y+1}",
                    type="revolute",
                )
                parent = etree.SubElement(joint_sw, "parent")
                parent.text = f"mass_x{idx_x}_y{idx_y}"
                child = etree.SubElement(joint_sw, "child")
                child.text = f"mass_x{idx_x-1}_y{idx_y+1}"

                physics = etree.SubElement(joint_sw, "physics")
                ode = etree.SubElement(physics, "ode")
                cfm_damping = etree.SubElement(ode, "cfm_damping")
                cfm_damping.text = "true"

                axis1 = etree.SubElement(joint_sw, "axis")
                xyz1 = etree.SubElement(axis1, "xyz")
                xyz1.text = "0 1 0"
                dynamics1 = etree.SubElement(axis1, "dynamics")
                # damping1 = etree.SubElement(dynamics1, "damping")
                # damping1.text = str(uni_joint_x_damping)
                spring_stiffness = etree.SubElement(dynamics1, "spring_stiffness")
                spring_stiffness.text = str(uni_joint_x_stiffness)
                spring_reference = etree.SubElement(dynamics1, "spring_reference")
                spring_reference.text = "0"


                pose = etree.SubElement(joint_sw, "pose", relative_to=f"mass_x{idx_x-1}_y{idx_y+1}")
                pose.text = f"{0.5*x_sep} {-0.5*y_sep} 0 0 0 {0.75*np.pi}"

            #check if north west diagonal connection is possible, if so, create joint
            if idx_x < num_elements_x - 1 and idx_y < num_elements_y - 1 and create_diag:
                num_springs_dampers += 1
                joint_nw = etree.SubElement(
                    robot,
                    "joint",
                    name=f"NW_x{idx_x}_y{idx_y}_to_x{idx_x+1}_y{idx_y+1}",
                    type="revolute",
                )
                parent = etree.SubElement(joint_nw, "parent")
                parent.text = f"mass_x{idx_x}_y{idx_y}"
                child = etree.SubElement(joint_nw, "child")
                child.text = f"mass_x{idx_x+1}_y{idx_y+1}"

                physics = etree.SubElement(joint_nw, "physics")
                ode = etree.SubElement(physics, "ode")
                cfm_damping = etree.SubElement(ode, "cfm_damping")
                cfm_damping.text = "true"

               

                
                axis1 = etree.SubElement(joint_nw, "axis")
                xyz1 = etree.SubElement(axis1, "xyz")
                xyz1.text = "0 1 0"
                dynamics1 = etree.SubElement(axis1, "dynamics")
                # damping1 = etree.SubElement(dynamics1, "damping")
                # damping1.text = str(uni_joint_x_damping)
                spring_stiffness = etree.SubElement(dynamics1, "spring_stiffness")
                spring_stiffness.text = str(uni_joint_x_stiffness)
                spring_reference = etree.SubElement(dynamics1, "spring_reference")
                spring_reference.text = "0"

                pose = etree.SubElement(joint_nw, "pose", relative_to=f"mass_x{idx_x+1}_y{idx_y+1}")
                pose.text = f"{-0.5*x_sep} {-0.5*y_sep} 0 0 0 {np.pi/4}"

# create a fixed joint between masses and world where ind_y = 0
for ind_x in range(num_elements_x):
    fixed_joint = etree.SubElement(
        robot,
        "joint",
        name=f"fixed_joint_mass_x{ind_x}_y{0}_to_world",
        type="fixed",
    )   
    parent = etree.SubElement(fixed_joint, "parent")
    parent.text = "world"
    child = etree.SubElement(fixed_joint, "child")
    # child.text = f"mass_x{num_elements_x-1}_y{0}"
    child.text = f"mass_x{ind_x}_y{0}"
    
    # fixed_joint2 = etree.SubElement(
    #     robot,
    #     "joint",
    #     name=f"fixed_joint_mass_x0_y{9}_to_world",
    #     type="fixed",
    # )   
    # parent = etree.SubElement(fixed_joint2, "parent")
    # parent.text = "world"
    # child = etree.SubElement(fixed_joint2, "child")
    # # child.text = f"mass_x{num_elements_x-1}_y{num_elements_y-1}"
    # child.text = f"mass_x{0}_y{0}"
 

print(f"the number of elements is {num_elements_x} and the number of sd elements are {num_springs_dampers}")
with open("spring_damper_mesh.sdf", "wb") as f:
    f.write(etree.tostring(sdf, pretty_print=True))

the number of elements is 10 and the number of sd elements are 162


In [ ]:
# spherical along x, and y, with revolute springs along all diagonals

skyweave_mass = 2.1  # kg

z_offset = 1.2  # m

num_elements_x = 10
length_x = 0.8 # m
x_sep = length_x / (num_elements_x - 1) # the separation between each consecutive mass element in the x axis 

num_elements_y = 10
length_y = 0.8  # m
y_sep = length_y / (num_elements_y - 1) # the separation between each consecutive mass element in the y axis

stiffness = 2
damping = 0

create_diag = True

uni_joint_x_stiffness = stiffness  # N/rad
uni_joint_y_stiffness = stiffness  # N/rad
uni_joint_x_damping = damping  # N/(rad/s)
uni_joint_y_damping = damping  # N/(rad/s)

num_elements = num_elements_x * num_elements_y
element_mass = skyweave_mass / num_elements

sdf = etree.Element("sdf", version="1.7")
robot = etree.SubElement(sdf, "model", name="spring_damper_mesh")
# pose = etree.SubElement(robot, "pose")
# pose.text = "0 0 0 0 0 0"
static = etree.SubElement(robot, "static")
static.text = "false"

# create a grid of point masses
pt_masses = []
for idx_y in range(num_elements_y):
    pt_masses.append([])
    for idx_x in range(num_elements_x):
        m = etree.SubElement(
            robot,
            "link",
            name=f"mass_x{idx_x}_y{idx_y}",
        )
        pose = etree.SubElement(m, "pose")
        pose.text = f"{(idx_x)*x_sep} {(idx_y)*y_sep} {z_offset} 0 0 0"
 
        inertial = etree.SubElement(m, "inertial")
        mass = etree.SubElement(inertial, "mass")
        mass.text = str(element_mass)
        inertia = etree.SubElement(inertial, "inertia")

        # define all inertia components and their values
        inertia_values = {
            "ixx": "1e-5", "ixy": "0", "ixz": "0",
            "iyy": "1e-5", "iyz": "0",
            "izz": "1e-5"
        }

        # create them in a loop
        for name, value in inertia_values.items():
            etree.SubElement(inertia, name).text = value

        collision = etree.SubElement(m, "collision", name=f"collision_x{idx_x}_y{idx_y}")
        geometry = etree.SubElement(collision, "geometry")
        sphere = etree.SubElement(geometry, "sphere")
        radius = etree.SubElement(sphere, "radius")
        radius.text = "0.001"

        visual = etree.SubElement(m, "visual", name=f"visual_x{idx_x}_y{idx_y}")
        geometry_v = etree.SubElement(visual, "geometry")
        sphere_v = etree.SubElement(geometry_v, "box")
        size_v = etree.SubElement(sphere_v, "size")
        size_v.text = "0.015 0.015 0.015"


        pt_masses[idx_y].append(m)

num_springs_dampers = 0
# link adjacent point masses with xy universal joints
for idx_y in range(num_elements_y):
    for idx_x in range(num_elements_x):
        # check if north connection is possible, if so create joint
        ## what are the conditions for a north connection to be possible?
        if idx_x < num_elements_x - 1:
            joint_north = etree.SubElement(
                robot,
                "joint",
                name=f"N_x{idx_x}_y{idx_y}_to_x{idx_x+1}_y{idx_y}",
                type="ball",
            )

            parent = etree.SubElement(joint_north, "parent")
            parent.text = f"mass_x{idx_x}_y{idx_y}"
            child = etree.SubElement(joint_north, "child")
            child.text = f"mass_x{idx_x+1}_y{idx_y}"

            pose = etree.SubElement(joint_north, "pose", relative_to=f"mass_x{idx_x+1}_y{idx_y}")
            pose.text = f"{-0.5*x_sep} 0 0 0 0 0"

        # check if west connection is possible, if so, create joint
        if idx_y < num_elements_y - 1:
            joint_west = etree.SubElement(
                robot,
                "joint",
                name=f"W_x{idx_x}_y{idx_y}_to_x{idx_x}_y{idx_y+1}",
                type="ball",
            )

            parent = etree.SubElement(joint_west, "parent")
            parent.text = f"mass_x{idx_x}_y{idx_y}"
            child = etree.SubElement(joint_west, "child")
            child.text = f"mass_x{idx_x}_y{idx_y+1}"

            pose = etree.SubElement(joint_west, "pose", relative_to=f"mass_x{idx_x}_y{idx_y+1}")
            pose.text = f"0 {-0.5*y_sep} 0 0 0 {np.pi/2}"

        # link diagonal point masses with shear springs attached to ball joints to prevent shear
        # # check if idx_x is even, and idx y is odd, or if idx_x is odd and idx_y is even
        if (idx_x + idx_y) % 2 == 1 or True: # NOTE we are not doing this anymore, as we want all diagonal connections
            # check if south west joint is possible
            if idx_x > 0 and idx_y < num_elements_y - 1 and create_diag:
                num_springs_dampers += 1
                joint_sw = etree.SubElement(
                    robot,
                    "joint",
                    name=f"SW_x{idx_x}_y{idx_y}_to_x{idx_x-1}_y{idx_y+1}",
                    type="ball",
                )
                parent = etree.SubElement(joint_sw, "parent")
                parent.text = f"mass_x{idx_x}_y{idx_y}"
                child = etree.SubElement(joint_sw, "child")
                child.text = f"mass_x{idx_x-1}_y{idx_y+1}"

                pose = etree.SubElement(joint_sw, "pose", relative_to=f"mass_x{idx_x-1}_y{idx_y+1}")
                pose.text = f"{0.5*x_sep} {-0.5*y_sep} 0 0 0 {0.75*np.pi}"

            #check if north west diagonal connection is possible, if so, create joint
            if idx_x < num_elements_x - 1 and idx_y < num_elements_y - 1 and create_diag:
                num_springs_dampers += 1
                joint_nw = etree.SubElement(
                    robot,
                    "joint",
                    name=f"NW_x{idx_x}_y{idx_y}_to_x{idx_x+1}_y{idx_y+1}",
                    type="ball",
                )
                parent = etree.SubElement(joint_nw, "parent")
                parent.text = f"mass_x{idx_x}_y{idx_y}"
                child = etree.SubElement(joint_nw, "child")
                child.text = f"mass_x{idx_x+1}_y{idx_y+1}"

                pose = etree.SubElement(joint_nw, "pose", relative_to=f"mass_x{idx_x+1}_y{idx_y+1}")
                pose.text = f"{-0.5*x_sep} {-0.5*y_sep} 0 0 0 {np.pi/4}"

# create a fixed joint between masses and world where ind_y = 0
for ind_x in range(1):
    fixed_joint = etree.SubElement(
        robot,
        "joint",
        name=f"fixed_joint_mass_x{ind_x}_y{0}_to_world",
        type="fixed",
    )   
    parent = etree.SubElement(fixed_joint, "parent")
    parent.text = "world"
    child = etree.SubElement(fixed_joint, "child")
    # child.text = f"mass_x{num_elements_x-1}_y{0}"
    child.text = f"mass_x{ind_x}_y{0}"
    
    # fixed_joint2 = etree.SubElement(
    #     robot,
    #     "joint",
    #     name=f"fixed_joint_mass_x0_y{1}_to_world",
    #     type="fixed",
    # )   
    # parent = etree.SubElement(fixed_joint2, "parent")
    # parent.text = "world"
    # child = etree.SubElement(fixed_joint2, "child")
    # # child.text = f"mass_x{num_elements_x-1}_y{num_elements_y-1}"
    # child.text = f"mass_x{0}_y{1}"
 
    # fixed_joint3 = etree.SubElement(
    #     robot,
    #     "joint",
    #     name=f"fixed_joint_mass_x{1}_y{1}_to_world",
    #     type="fixed",
    # )   
    # parent = etree.SubElement(fixed_joint3, "parent")
    # parent.text = "world"
    # child = etree.SubElement(fixed_joint3, "child")
    # # child.text = f"mass_x{num_elements_x-1}_y{num_elements_y-1}"
    # child.text = f"mass_x{1}_y{0}"
 

print(f"the number of elements is {num_elements_x} and the number of sd elements are {num_springs_dampers}")
with open("spring_damper_mesh.sdf", "wb") as f:
    f.write(etree.tostring(sdf, pretty_print=True))

the number of elements is 10 and the number of sd elements are 162
